# Echo-A 1B5 (Memory Finetune)
This continues off from `Echo-A-1B5-basemodel.ipynb` to perform the full memory finetune & testing process

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Prepare the dataset

Prepare and preload the finetuning process dataset

In [13]:
# Ensure NPM dependencies are installed
!cd memory_dataset && npm install . && echo "## NPM dependencies installed ##"

# Build the dataset
!cd memory_dataset && ./gen_dataset_first_2k.sh

m#########⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠴ idealTree: timing idealTree Completed in 11ms
up to date, audited 3 packages in 423ms

found 0 vulnerabilities
## NPM dependencies installed ##
## Generating word reptition dataset ##
Generated JSONL file with 20000 samples at /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/experiment/memory/memory_dataset/dataset/word-2-count.jsonl
Generated JSONL file with 20000 samples at /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/experiment/memory/memory_dataset/dataset/word-5-count.jsonl
Generated JSONL file with 20000 samples at /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/experiment/memory/memory_dataset/dataset/word-10-count.jsonl
Generated JSONL file with 20000 samples at /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/experiment/memory/memory_dataset/dataset/word-20-count.jsonl
Generated JSONL file with 8000 samples at /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/ex

In [14]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-A-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
TRAINER_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/experiment/memory
TRAINER_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v4neo


## Stage 3 : Simple Memory finetuning

In [15]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/Echo-A-1B5-mem-finetune/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 252.50it/s]
Dataset json downloaded and prepared to /home/picocreator/.cache/huggingface/datasets/json/default-552a348358a7dd4c/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 50.41it/s]
mkdir: invalid option -- 'f'                                                    
Try 'mkdir --help' for more information.


In [16]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-1 (bs=128, train-ctx=2048)" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=2048

[2023-07-10 01:54:13,137] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1148776073
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1148776073
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230710_015415-jf9erm5p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run Echo-A-1B5 - Mem-Finetune-1 (bs=128, train-ctx=2048)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/jf9erm5p
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorc

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    && python export_checkpoint.py "../checkpoint/Echo-A-1B5-mem-finetune/last.ckpt"

In [ ]:
# Lets move and save this model
!cd "{TRAINER_DIR}" && cp ./checkpoint/Echo-A-1B5-mem-finetune/last.ckpt/rwkv_model.pth ./model/Echo-A-1B5-Stage3.pth
!cd "{TRAINER_DIR}" && ls -alh ./model/Echo-A-1B5-Stage3.pth

In [ ]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/Echo-A-1B5-Stage3.pth "cuda fp32"